# DS2002 Midterm
Modern Data Warehousing and Analytics solutions frequently use languages like Python or Scala to extract data from numerous sources, including relational database management systems, NoSQL database systems, real-time streaming endpoints and Data Lakes.  These languages can then be used to perform many types of transformation before then loading the data into a variety of destinations including file systems and data warehouses. This data can then be consumed by data scientists or business analysts.


### Prerequisites:
This notebook uses the SqlAlchemy database connectivity library to connect to MySQL databases; therefore, you must have first installed that libary into your python environment by executing the following command in a Terminal window.

- `python -m pip install sqlalchemy`

#### Import the Necessary Libraries

## Implementing MySQL

In [1]:
from platform import python_version
print(f"Running Python Version: {python_version()}")
import pymysql
print(f"Running PyMySQL Version: {pymysql.__version__}")
import sqlalchemy
print(f"Running SQL Alchemy Version: {sqlalchemy.__version__}")


import os
import numpy
import pandas as pd
from sqlalchemy import create_engine

Running Python Version: 3.11.3
Running PyMySQL Version: 1.4.6
Running SQL Alchemy Version: 1.4.39


#### Declare & Assign Connection Variables for the MySQL Server & Databases with which You'll be Working 

In [2]:
host_name = "localhost"
port = "3306"
user_id = "root"
pwd = "Passw0rd123"

src_dbname = "classicmodels"
dst_dbname = "classicmodels_dw2"

#### Define Functions for Getting Data From and Setting Data Into Databases

In [3]:
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

#### Create the New Data Warehouse database, and to Use it, Switch the Connection Context.
Clearly, you won't get very far without having a database to work with. Here we demonstrate how we can *drop* a database if it already exists, and then *create* the new **classicmodels_dw2** database and *use* it as the target of all subsequent operations.

In [4]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)
connection = sqlEngine.connect()

connection.execute(f"DROP DATABASE IF EXISTS `{dst_dbname}`;")
connection.execute(f"CREATE DATABASE `{dst_dbname}`;")
connection.execute(f"USE {dst_dbname};")

connection.close()

### 1.0. Create & Populate the Dimension Tables
In any extract-transform-load (ETL) process used to populate a multi-dimensional data warehouse database it is necessary to populate the Dimension tables before attempting to populate the Fact table(s). This is because rows in the Fact table(s) will reference surrogate primary key values from the Dimension tables. If the primary key values in the Dimension tables either do not exist, or do not reflect the current state of the dimension, then the attempt to load the Fact table(s) will fail.


#### 1.1. Extract Data from the Source Database Tables
Fetch data for each dimension table (e.g., customers, employees, products, shippers) from the **classicmodels** database using the **get_dataframe()** function.

In [5]:
# Products
sql_products = "SELECT * FROM classicmodels.products;"
df_products = get_dataframe(user_id, pwd, host_name, src_dbname, sql_products)
df_products.head(2)

,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.7
1,S10_1949,1952 Alpine Renault 1300,Classic Cars,1:10,Classic Metal Creations,Turnable front wheels; steering function; deta...,7305,98.58,214.3


#### 1.2. Create the Date Dimension Table
At this point, we have to **execute the script from Lab 2c** that creates and populates a **Date Dimension** table.  Be certain to target this script to the new data warehouse database we just created **(classicmodels_dw2)**.  Later in this notebook we will integrate the **dim_date** table with the fact table by performing **lookup operations** to retrieve the surrogate primary keys from the **dim_date** table that correspond with each **date** typed column in the fact table (e.g., order_date).

#### 1.3. Perform Any Necessary Transformations
Pandas DataFrames enable extensive data modification capabilities. Here we will start by simply dropping features (columns) that we don't believe provide any real value to our analytics solution. Examples include columns having a high percentage of NULL values, columns having large amounts of free-text, and columns having binary large object (BLOB) data such as images or other documents. Then, we will rename the primary key column from the source (id) to serve as the business key for future lookup operations. Finally, we will *insert* a new primary key column that contains and ever-increasing numeric value.  It should be named after the entity (e.g., customer, product) followed by "**_key**" to conform with data warehouse design standards.

In [6]:
# Products
drop_cols = ['productDescription']
df_products.drop(drop_cols, axis=1, inplace=True)
df_products.insert(0, "product_key", range(1, df_products.shape[0]+1))
df_products.head(2)

,product_key,productCode,productName,productLine,productScale,productVendor,quantityInStock,buyPrice,MSRP
0,1,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,7933,48.81,95.7
1,2,S10_1949,1952 Alpine Renault 1300,Classic Cars,1:10,Classic Metal Creations,7305,98.58,214.3


#### 1.4. Load the Transformed DataFrames into the New Data Warehouse by Creating New Tables
Here we demonstrate how an iterable data structure can be created containing the values needed to correctly create and populate the new dimension tables. If you inspect this code listing carefully, you'll notice that it's a **list** containing a **set** *(or vector)* for each dimension table. Each **set** then contains the *table_name* we need to assign to the table, the *pandas DataFrame* we crafted to define & populate the table, and the name we need to assign to the *primary_key* column.  With this *list of sets* defined, we can then call our **set_dataframe( )** function from within a **for *loop*** to create each *dimension* table.

In [7]:
db_operation = "insert"
tables = [('dim_products', df_products, 'product_key')]

In [8]:
for table_name, dataframe, primary_key in tables:
    set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

### 2.0. Create & Populate the Fact Table
Here we will learn two approaches to creating the *fact_orders* fact table. The first approach demonstrates that a carefully crafted SQL SELECT statement can be used to perform this task... *but what fun would that be.* Seriously though, this approach is quick and effect if you already have the query, but what if you didn't have the opportunity to view and work with the data beforehand?  What's more, you may be required to combine data from multiple sources, some of which may not be relational database management systems. Then, a simple SQL query won't do!  You would need to load the data from the various sources (e.g., database tables, CSV or JSON files, NoSQL document collections, API stream data) and then combine them into a single dataframe that you could then use to create a new database table. For this reason we'll see how we can retrieve the data, but we won't bother to use it for creating a new table... we already know how to do that using the **set_dataframe( )** function anyway.


### Implement the solution using Pandas DataFrames to craft the table


#### 2.1. Get all the data from each of the tables involved

In [9]:
sql_orders = "SELECT * FROM classicmodels.orders;"
df_orders = get_dataframe(user_id, pwd, host_name, src_dbname, sql_orders)
#df_orders.rename(columns={"id":"order_id"}, inplace=True)
df_orders.head(2)

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,363
1,10101,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128


In [10]:
sql_orderdetails = "SELECT * FROM classicmodels.orderdetails;"
df_orderdetails = get_dataframe(user_id, pwd, host_name, src_dbname, sql_orderdetails)
df_orderdetails.head(2)

,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,10100,S18_1749,30,136.00,3
1,10100,S18_2248,50,55.09,2


#### 2.4. Join the Orders and OrderDetails DataFrames
In this step we can now easily join the *orders* and *order_details* dataframes. Since each **order** (the *left* dataframe) can have many **order details** (the *right* dataframe), we'll need to implement a **right** *outer join* **on** the *order_id* column.

In [11]:
df_fact_orders = pd.merge(df_orders, df_orderdetails, on='orderNumber', how='inner')
df_fact_orders.head(2)

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,363,S18_1749,30,136.00,3
1,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,363,S18_2248,50,55.09,2


In [12]:
df_fact_orders.shape

(2996, 11)

#### 2.5. Lookup the Primary Keys from the Dimension Tables
Just as we did in **Lab 1**, we need to establish **foreign key relationships** between the newly-crafted **Fact table** and each of the **Dimension tables**.

##### 2.5.1. First, fetch the Surrogate Primary Key and the Business Key from each of the Dimension tables using the **get_dataframe()** function.

In [13]:
# Products
sql_dim_products = "SELECT product_key, productCode FROM classicmodels_dw2.dim_products;"
df_dim_products = get_dataframe(user_id, pwd, host_name, src_dbname, sql_dim_products)
df_dim_products.head(2)

,product_key,productCode
0,1,S10_1678
1,2,S10_1949


##### 2.5.2. Next, using the Business Keys, lookup the corresponding Surrogate Primary Key values in the Dimension tables

In [14]:
# Repeat for the Products dimension
df_fact_orders = pd.merge(df_fact_orders, df_dim_products, on='productCode', how='inner')
df_fact_orders.drop(['productCode'], axis=1, inplace=True)
df_fact_orders.head(2)

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber,quantityOrdered,priceEach,orderLineNumber,product_key
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,363,30,136.0,3,23
1,10110,2003-03-18,2003-03-24,2003-03-20,Shipped,None,187,42,153.0,7,23


##### 2.5.3. Lookup the DateKeys from the Date Dimension Table.
First, fetch the Surrogate Primary Key (date_key) and the Business Key (full_date) from the Date Dimension table using the **get_dataframe()** function. Be certain to cast the **full_date** column to the **datetime64[ns]** data type using the **.astype()** function that is native to Pandas DataFrame columns. Also, extract the **date** portion using the **.dt.date** attribute of the **datetime64[ns]** datatype.

In [15]:
sql_dim_date = "SELECT date_key, full_date FROM classicmodels_dw2.dim_date;"
df_dim_date = get_dataframe(user_id, pwd, host_name, src_dbname, sql_dim_date)
df_dim_date.full_date = df_dim_date.full_date.astype('datetime64[ns]').dt.date
df_dim_date.head(2)

,date_key,full_date
0,20000101,2000-01-01
1,20000102,2000-01-02


Next, for each **date** typed column in the Fact table, lookup the corresponding Primary Key column. Be certain to cast each **date** column to the **datetime64[ns]** data type using the **.astype()** function that's native to Pandas DataFrame columns. Also, extract the **date** portion using the **.dt.date** attribute.

In [16]:
# Lookup the Surrogate Primary Key (date_key) that Corresponds to the "orderDate" Column.
df_dim_orderDate = df_dim_date.rename(columns={"date_key" : "orderDate_key", "full_date" : "orderDate"})
df_fact_orders.orderDate = df_fact_orders.orderDate.astype('datetime64[ns]').dt.date

df_fact_orders = pd.merge(df_fact_orders, df_dim_orderDate, on='orderDate', how='left')
df_fact_orders.drop(['orderDate'], axis=1, inplace=True)
df_fact_orders.head(2)

,orderNumber,requiredDate,shippedDate,status,comments,customerNumber,quantityOrdered,priceEach,orderLineNumber,product_key,orderDate_key
0,10100,2003-01-13,2003-01-10,Shipped,None,363,30,136.0,3,23,NaN
1,10110,2003-03-24,2003-03-20,Shipped,None,187,42,153.0,7,23,NaN


In [17]:
# Lookup the Surrogate Primary Key (date_key) that Corresponds to the "requiredDate" Column.
df_dim_requiredDate = df_dim_date.rename(columns={"date_key" : "requiredDate_key", "full_date" : "requiredDate"})
df_fact_orders.requiredDate = df_fact_orders.requiredDate.astype('datetime64[ns]').dt.date

df_fact_orders = pd.merge(df_fact_orders, df_dim_requiredDate, on='requiredDate', how='left')
df_fact_orders.drop(['requiredDate'], axis=1, inplace=True)
df_fact_orders.head(2)

,orderNumber,shippedDate,status,comments,customerNumber,quantityOrdered,priceEach,orderLineNumber,product_key,orderDate_key,requiredDate_key
0,10100,2003-01-10,Shipped,None,363,30,136.0,3,23,NaN,NaN
1,10110,2003-03-20,Shipped,None,187,42,153.0,7,23,NaN,NaN


In [18]:
# Lookup the Surrogate Primary Key (date_key) that Corresponds to the "shippedDate" Column.
df_dim_shippedDate = df_dim_date.rename(columns={"date_key" : "shippedDate_key", "full_date" : "shippedDate"})
df_fact_orders.shippedDate = df_fact_orders.shippedDate.astype('datetime64[ns]').dt.date

df_fact_orders = pd.merge(df_fact_orders, df_dim_shippedDate, on='shippedDate', how='left')
df_fact_orders.drop(['shippedDate'], axis=1, inplace=True)
df_fact_orders.head(2)

,orderNumber,status,comments,customerNumber,quantityOrdered,priceEach,orderLineNumber,product_key,orderDate_key,requiredDate_key,shippedDate_key
0,10100,Shipped,None,363,30,136.0,3,23,NaN,NaN,NaN
1,10110,Shipped,None,187,42,153.0,7,23,NaN,NaN,NaN


#### 2.6. Perform any Additional Transformations
In this step we can prepare the DataFrame so that it defines exactly what we want to see created in the database.  Issues may include dropping unwanted columns, reordering the columns, and in our case, creating a new column to serve as the primary key.

In [19]:
df_fact_orders.insert(0, "fact_order_key", range(1, df_fact_orders.shape[0]+1))
df_fact_orders.head(2)

,fact_order_key,orderNumber,status,comments,customerNumber,quantityOrdered,priceEach,orderLineNumber,product_key,orderDate_key,requiredDate_key,shippedDate_key
0,1,10100,Shipped,None,363,30,136.0,3,23,NaN,NaN,NaN
1,2,10110,Shipped,None,187,42,153.0,7,23,NaN,NaN,NaN


#### 2.7. Write the DataFrame Back to the Database

In [20]:
table_name = "fact_orders"
primary_key = "fact_order_key"
db_operation = "insert"

set_dataframe(user_id, pwd, host_name, dst_dbname, df_fact_orders, table_name, primary_key, db_operation)

## Using Python to Integrate MongoDB Data into an ETL Process

### Prerequisites:
This notebook uses the PyMongo database connectivity library to connect to MySQL databases; therefore, you must have first installed that libary into your python environment by executing the following command in a Terminal window.

- `python -m pip install pymongo[srv]`

#### Import the Necessary Libraries

In [21]:
import os
import json
import numpy
import datetime
import certifi
import pandas as pd

import pymongo
import sqlalchemy
from sqlalchemy import create_engine

In [22]:
print(f"Running SQL Alchemy Version: {sqlalchemy.__version__}")
print(f"Running PyMongo Version: {pymongo.__version__}")

Running SQL Alchemy Version: 1.4.39
Running PyMongo Version: 4.6.3


#### Declare & Assign Connection Variables for the MongoDB Server, the MySQL Server & Databases with which You'll be Working 

In [23]:
mysql_args = {
    "uid" : "root",
    "pwd" : "Passw0rd123",
    "hostname" : "localhost",
    "dbname" : "classicmodels_dw2"
}

# The 'cluster_location' must either be "atlas" or "local".
mongodb_args = {
    "user_name" : "",
    "password" : "password",
    "cluster_name" : "cluster_name",
    "cluster_subnet" : "xxxxx",
    "cluster_location" : "local", # "local"
    "db_name" : "classicmodels_customers"
}

#### Define Functions for Getting Data From and Setting Data Into Databases

In [24]:
def get_sql_dataframe(sql_query, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe
    

def set_dataframe(df, table_name, pk_column, db_operation, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()


def get_mongo_client(**args):
    '''Validate proper input'''
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the cluster_location parameter.")
    
    else:
        if args["cluster_location"] == "atlas":
            connect_str = f"mongodb+srv://{args['user_name']}:{args['password']}@"
            connect_str += f"{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net"
            client = pymongo.MongoClient(connect_str, tlsCAFile=certifi.where())
            
        elif args["cluster_location"] == "local":
            client = pymongo.MongoClient("mongodb://localhost:27017/")
        
    return client


def get_mongo_dataframe(mongo_client, db_name, collection, query):
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    mongo_client.close()
    
    return dframe


def set_mongo_collections(mongo_client, db_name, data_directory, json_files):
    db = mongo_client[db_name]
    
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(data_directory, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)
        
    mongo_client.close()

#### Populate MongoDB with Source Data
You only need to run this cell once; however, the operation is *idempotent*.  In other words, it can be run multiple times without changing the end result.

In [25]:
client = get_mongo_client(**mongodb_args)

# Gets the path of the Current Working Directory for this Notebook,
# and then Appends the 'data' directory.
data_dir = os.path.join(os.getcwd(), 'data')

json_files = {"customers" : 'classicmodels_customers.json'}

set_mongo_collections(client, mongodb_args["db_name"], data_dir, json_files)         

### 1.0. Create and Populate the New Dimension Tables
#### 1.1. Extract Data from the Source MongoDB Collections Into DataFrames

In [26]:
client = get_mongo_client(**mongodb_args)

query = {} # Select all elements (columns), and all documents (rows).
collection = "customers"

df_customers = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_customers.head(2)

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",None,Nantes,None,44000,France,1370.0,21000.0
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,None,Las Vegas,NV,83030,USA,1166.0,71800.0


#### 1.3. Perform Any Necessary Transformations to the DataFrames

In [27]:
# 1. Rename the "id" column to reflect the entity as it will serve as the business key for lookup operations
#df_customers.rename(columns={"id":"invoice_id"}, inplace=True)

# 2. Since there are no values in the 'due_date' column go ahead and drop it.
df_customers.drop(['phone','addressLine2','state'], axis=1, inplace=True)

# 3. Insert a new column, with an ever-incrementing numeric value, to serve as the primary key.
df_customers.insert(0, "customer_key", range(1, df_customers.shape[0]+1))
df_customers.head(2)

,customer_key,customerNumber,customerName,contactLastName,contactFirstName,addressLine1,city,postalCode,country,salesRepEmployeeNumber,creditLimit
0,1,103,Atelier graphique,Schmitt,Carine,"54, rue Royale",Nantes,44000,France,1370.0,21000.0
1,2,112,Signal Gift Stores,King,Jean,8489 Strong St.,Las Vegas,83030,USA,1166.0,71800.0


#### 1.3. Load the Transformed DataFrames into the New Data Warehouse by Creating New Tables

Here we will call our **set_dataframe( )** function to create each dimension table. This function expects a number of parameters including the usual connection information (e.g., user_id, password, MySQL server name and database), the *table_name* we need to assign to the table, the *pandas DataFrame* we crafted to define & populate the table, the *name* of the column we wish to designate as the *primary_key* column, and finally, the database operation (insert or update). 

In [28]:
dataframe = df_customers
table_name = 'dim_customers'
primary_key = 'customer_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

#### 1.4. Validate that the New Dimension Tables were Created.

In [29]:
sql_customers = "SELECT * FROM classicmodels_dw2.dim_customers;"
df_dim_customers = get_sql_dataframe(sql_customers, **mysql_args)
df_dim_customers.head(2)

,customer_key,customerNumber,customerName,contactLastName,contactFirstName,addressLine1,city,postalCode,country,salesRepEmployeeNumber,creditLimit
0,1,103,Atelier graphique,Schmitt,Carine,"54, rue Royale",Nantes,44000,France,1370.0,21000.0
1,2,112,Signal Gift Stores,King,Jean,8489 Strong St.,Las Vegas,83030,USA,1166.0,71800.0


#### Lookup primary key

In [30]:
df_fact_orders = pd.merge(df_fact_orders, df_dim_customers, on='customerNumber', how='inner')
df_fact_orders.drop(['customerNumber'], axis=1, inplace=True)
df_fact_orders.head(2)

,fact_order_key,orderNumber,status,comments,quantityOrdered,priceEach,orderLineNumber,product_key,orderDate_key,requiredDate_key,...,customer_key,customerName,contactLastName,contactFirstName,addressLine1,city,postalCode,country,salesRepEmployeeNumber,creditLimit
0,1,10100,Shipped,None,30,136.00,3,23,NaN,NaN,...,86,Online Diecast Creations Co.,Young,Dorothy,2304 Long Airport Avenue,Nashua,62005,USA,1216.0,114200.0
1,26,10100,Shipped,None,50,55.09,2,27,NaN,NaN,...,86,Online Diecast Creations Co.,Young,Dorothy,2304 Long Airport Avenue,Nashua,62005,USA,1216.0,114200.0


# Using CSV file

- `python -m pip install sqlalchemy`

#### Import the Necessary Libraries

In [31]:
import os
import pandas as pd

In [32]:
data_dir = os.path.join(os.getcwd(), 'data')
data_file = os.path.join(data_dir, 'classicmodels_productlines.csv')

df_productlines = pd.read_csv(data_file, header=0, index_col=False)
df_productlines.head()

,productLine,textDescription,htmlDescription,image
0,Classic Cars,Attention car enthusiasts: Make your wildest c...,NaN,NaN
1,Motorcycles,Our motorcycles are state of the art replicas ...,NaN,NaN
2,Planes,"Unique, diecast airplane and helicopter replic...",NaN,NaN
3,Ships,The perfect holiday or anniversary gift for ex...,NaN,NaN
4,Trains,Model trains are a rewarding hobby for enthusi...,NaN,NaN


#### Drop features, Form necessary transformations

In [33]:
df_productlines.drop(['htmlDescription','image'], axis=1, inplace=True)
df_productlines.head(2)

,productLine,textDescription
0,Classic Cars,Attention car enthusiasts: Make your wildest c...
1,Motorcycles,Our motorcycles are state of the art replicas ...


In [34]:
df_productlines.insert(0, "productline_key", range(1, df_productlines.shape[0]+1))
df_productlines.head(2)

,productline_key,productLine,textDescription
0,1,Classic Cars,Attention car enthusiasts: Make your wildest c...
1,2,Motorcycles,Our motorcycles are state of the art replicas ...


Load the Transformed DataFrames into the New Data Warehouse by Creating New Tables

In [35]:
dataframe = df_productlines
table_name = 'dim_productlines'
primary_key = 'productline_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

 Extract Data from the Source Database Tables

In [36]:
sql_productlines = "SELECT * FROM classicmodels_dw2.dim_productlines;"
df_dim_productlines = get_sql_dataframe(sql_productlines, **mysql_args)
df_dim_productlines.head(2)

,productline_key,productLine,textDescription
0,1,Classic Cars,Attention car enthusiasts: Make your wildest c...
1,2,Motorcycles,Our motorcycles are state of the art replicas ...


#### Lookup primary keys that correspond to "products" and "productLine"
As discussed in Lecture, classicmodels has a connection between products and productlines that can be explored in this project.

In [37]:
df_fact_orders = pd.merge(df_products, df_dim_productlines, on='productLine', how='inner')
df_fact_orders.drop(['productLine'], axis=1, inplace=True)
df_fact_orders.head(2)

,product_key,productCode,productName,productScale,productVendor,quantityInStock,buyPrice,MSRP,productline_key,textDescription
0,1,S10_1678,1969 Harley Davidson Ultimate Chopper,1:10,Min Lin Diecast,7933,48.81,95.70,2,Our motorcycles are state of the art replicas ...
1,3,S10_2016,1996 Moto Guzzi 1100i,1:10,Highway 66 Mini Classics,6625,68.99,118.94,2,Our motorcycles are state of the art replicas ...


## SQL Queries
To demonstrate the viability of your solution, author SQL SELECT statements that have data from at least 3 tables (fact table plus two dimensions), and performs some type of aggregation

#### I authored two SQL queries. The first query aggregates data from fact orders table and products table. The second query aggregates data from products table and productlines table.

In [48]:
sql_test_1 = """
    SELECT p.productName AS 'Product Name'
        , FORMAT(SUM(fo.quantityOrdered),0) AS 'Total Quantity'
    FROM classicmodels_dw2.fact_orders AS fo
    INNER JOIN classicmodels_dw2.dim_products AS p
    ON fo.product_key = p.product_key
    GROUP BY p.productName;
"""

df_test_1 = get_dataframe(user_id, pwd, host_name, src_dbname, sql_test_1)
df_test_1.head()

,Product Name,Total Quantity
0,1917 Grand Touring Sedan,918
1,1911 Ford Town Car,832
2,1932 Alfa Romeo 8C2300 Spider Sport,866
3,1936 Mercedes Benz 500k Roadster,824
4,1932 Model A Ford J-Coupe,957


In [61]:
sql_test_2 = """
    SELECT pl.productLine AS 'Product Line'
        , FORMAT(SUM(p.quantityInStock),0) AS 'Quantity In Stock'
    FROM classicmodels_dw2.dim_products AS p
    INNER JOIN classicmodels_dw2.dim_productlines AS pl
    ON pl.productline_key = p.product_key
    GROUP BY pl.productLine;
"""

df_test_2 = get_dataframe(user_id, pwd, host_name, src_dbname, sql_test_2)
df_test_2.head()

,Product Line,Quantity In Stock
0,Classic Cars,"7,933"
1,Motorcycles,"7,305"
2,Planes,"6,625"
3,Ships,"5,582"
4,Trains,"3,252"
